In [25]:
import pandas as pd
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm

In [29]:
#Path
#quant_table_path = '/mnt/c/Users/quirosgu/Desktop/Huber/RH119_P1_MeOH_quant.csv'#here put the path to the folder where your quant table is.
metadata_table_path ='/mnt/c/Users/quirosgu/Documents/GitHub/Immuno_plants/data/Inmuno_metadata_final.tsv' #here put the path to the folder where your quant table is.
final_path ='/mnt/c/Users/quirosgu/Documents/GitHub/Immuno_plants/data/'#here put the path to the folder where your want to save your tables and htmls
#experiment = 'RH119' #put the unique code to recognize this set of data

In [30]:
#read the metadata table
metadata_df = pd.read_csv(metadata_table_path, sep='\t')
metadata_df.head()

,ID,Family,Genus,Species,Part,matched_name,taxon.synonyms,NCBITaxonomy,ott_id,query_otol_kingdom,query_otol_phylum,query_otol_class,query_otol_order,query_otol_family,query_otol_genus,query_otol_species
0,Acacia hayesii|L,fabaceae/mim.,Acacia,acacia hayesii,L,Acacia hayesii,"['Acacia acanthophylla', 'Acacia telensis', 'S...",1504162|Acacia hayesii,5785742.0,Archaeplastida,Streptophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,Acacia hayesii
1,Acalypha arvensis|WP,euphorbiaceae,Acalypha,acalypha arvensis,WP,Acalypha arvensis,"['Acalypha arvensis', 'Acalypha arvensis belan...",NaN,3910407.0,Archaeplastida,Streptophyta,Magnoliopsida,Malpighiales,Euphorbiaceae,Acalypha,Acalypha aristata
2,Acrocomia aculeata|L,arecaceae,Acrocomia,acrocomia aculeata,L,Acrocomia aculeata,"['Acrocomia antiguana', 'Acrocomia antioquiens...",169987|Acrocomia aculeata,817259.0,Archaeplastida,Streptophyta,Magnoliopsida,Arecales,Arecaceae,Acrocomia,Acrocomia aculeata
3,Acrostichum aureum|L,pteridaceae,Acrostichum,acrostichum aureum,L,Acrostichum aureum,"['Acrostichum aureum hirsutum', 'Acrostichum a...",29594|Acrostichum aureum,190945.0,Archaeplastida,Streptophyta,Polypodiopsida,Polypodiales,Pteridaceae,Acrostichum,Acrostichum aureum
4,Adelia triloba|AE,euphorbiaceae,Adelia,adelia triloba,AE,Adelia triloba,['Ricinella triloba'],353064|Adelia triloba,856043.0,Archaeplastida,Streptophyta,Magnoliopsida,Malpighiales,Euphorbiaceae,Adelia,Adelia triloba


In [31]:
dff = metadata_df.groupby(['query_otol_family']).count()
dff.shape

(109, 15)

In [32]:

# Reduce DataFrame to unique values in 'Column1'
unique_families = metadata_df['query_otol_family'].unique()
unique_genus = metadata_df['query_otol_genus'].unique()
unique_species = metadata_df['query_otol_species'].unique()

# Create a new DataFrame with only the unique values
dff = pd.DataFrame({'Unique_Families': unique_families})
dfg = pd.DataFrame({'Unique_Genus': unique_genus})
dfs = pd.DataFrame({'Unique_Species': unique_species})

sizef = dff.shape[0]
sizeg = dfg.shape[0]
sizes = dfs.shape[0]

print(f"Number of families: {sizef}")
print(f"Number of genus: {sizeg}")
print(f"Number of species: {sizes}")


Number of families: 109
Number of genus: 264
Number of species: 300


In [33]:
# ocurrences of each family, genus and species:

# Count the occurrences of each value in ''
family_counts = metadata_df['query_otol_family'].value_counts()
family_counts= family_counts.reset_index()
family_counts.columns = ['query_otol_family', 'Count']
family_counts.to_csv(final_path+'family_counts.tsv', sep='\t')

genus_counts = metadata_df['query_otol_genus'].value_counts()
genus_counts= genus_counts.reset_index()
genus_counts.columns = ['query_otol_genus', 'Count']
genus_counts.to_csv(final_path+'genus_counts.tsv', sep='\t')

species_counts = metadata_df['query_otol_species'].value_counts()
species_counts= species_counts.reset_index()
species_counts.columns = ['query_otol_species', 'Count']
species_counts.to_csv(final_path+'speciesfamily_counts.tsv', sep='\t')

family_counts

,query_otol_family,Count
0,Fabaceae,33
1,Rubiaceae,24
2,Asteraceae,17
3,Poaceae,17
4,Moraceae,13
...,...,...
104,Chloranthaceae,1
105,Calceolariaceae,1
106,Gunneraceae,1
107,Nyctaginaceae,1


### 1) Process the quantitative data

In [3]:
#recover correlation groups 
correlation_groups_df = pd.read_csv(quant_table_path, sep=',')#,  index_col='row ID')
correlation_groups_df.rename(columns = lambda x: x.replace(' Peak area', ''),inplace=True)
correlation_groups_df.drop(list(correlation_groups_df.filter(regex = 'Unnamed: ')), axis = 1, inplace = True) 
correlation_groups_df.drop(['row ion mobility', ''
    'row ion mobility unit', 'row CCS', 
    'correlation group ID', 'auto MS2 verify',
    'identified by n=', 'partners'], axis=1, inplace=True)
correlation_groups_df.rename(columns={'best ion': 'adduct (ion identity)', 'neutral M mass':'neutral mass (ion identity)', 'row retention time':'retention time (min)' }, inplace=True)
correlation_groups_df['retention time (min)'] = correlation_groups_df['retention time (min)'].round(decimals=2)
correlation_groups_df['row m/z'] = correlation_groups_df['row m/z'].round(decimals=4)

#complete correlation groups
correlation_groups_df['annotation network number'] = correlation_groups_df['annotation network number'].fillna(correlation_groups_df['row ID'].apply(str) + 'x')

quant_df = correlation_groups_df.copy()
quant_df.drop(['row ID', 'row m/z', 'retention time (min)', 'adduct (ion identity)','neutral mass (ion identity)'], axis=1, inplace=True)

#aggregate metadata columns according to correlations groups
correlation_groups_df = correlation_groups_df.iloc[:, :6]
agg_func = {'retention time (min)': 'mean', 'row m/z': set,  'adduct (ion identity)': set, 'row ID': set, 'neutral mass (ion identity)': 'max'}
dfcg = correlation_groups_df.groupby('annotation network number', as_index=False).agg(agg_func)
dfcg[['adduct (ion identity)', 'row ID']] = dfcg[['adduct (ion identity)', 'row ID']].astype(str)  

# reduced quant_df and groupby annotation network number
dfq = quant_df.groupby('annotation network number', as_index=False).max()

#add the metadata of the annotation groups
dff = pd.merge(dfq, dfcg[['annotation network number', 'retention time (min)', 'row m/z', 'row ID', 'adduct (ion identity)','neutral mass (ion identity)']], how ='left', left_on = 'annotation network number', right_on='annotation network number')
dff.fillna({'adduct (ion identity)': 'not available', 'neutral mass (ion identity)': 'not available'}, inplace=True)
dff.head(2)

,annotation network number,20230325_LFX_RH_3_RH119_P1_SPEMeOH_A02_1_Pos.mzML,20230325_LFX_RH_4_RH119_P1_SPEMeOH_A03_1_Pos.mzML,20230325_LFX_RH_13_RH119_P1_SPEMeOH_A12_1_Pos.mzML,20230325_LFX_RH_2_RH119_P1_SPEMeOH_A01_1_Pos.mzML,20230325_LFX_RH_5_RH119_P1_SPEMeOH_A04_1_Pos.mzML,20230325_LFX_RH_7_RH119_P1_SPEMeOH_A06_1_Pos.mzML,20230325_LFX_RH_9_RH119_P1_SPEMeOH_A08_1_Pos.mzML,20230325_LFX_RH_10_RH119_P1_SPEMeOH_A09_1_Pos.mzML,20230325_LFX_RH_8_RH119_P1_SPEMeOH_A07_1_Pos.mzML,...,20230325_LFX_RH_98_RH119_P1_SPEMeOH_H09_1_Pos.mzML,20230325_LFX_RH_100_RH119_P1_SPEMeOH_H11_1_Pos.mzML,20230325_LFX_RH_92_RH119_P1_SPEMeOH_H03_1_Pos.mzML,20230325_LFX_RH_97_RH119_P1_SPEMeOH_H08_1_Pos.mzML,20230325_LFX_RH_91_RH119_P1_SPEMeOH_H02_1_Pos.mzML,retention time (min),row m/z,row ID,adduct (ion identity),neutral mass (ion identity)
0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,113568.65,0.0,...,0.0,0.0,0.0,419083.62,161457.230,0.825,"{293.0633, 253.07}","{1, 2}","{'[M-H2O+H]+', '[M+Na]+'}",270.07394
1,1.0,52700.91,0.0,0.0,0.0,95447.28,90274.17,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,64374.223,0.930,"{793.2298, 771.2478}","{9, 10}","{'[M+H]+', '[M+Na]+'}",770.24056


In [4]:
#read the metadata table
metadata_df = pd.read_csv(metadata_table_path, sep=',')
metadata_df.head(2)

,filename,Type,Strain,Rep,full_name
0,20230325_LFX_RH_10_RH119_P1_SPEMeOH_A09_1_Pos....,blank,1239,rep2,blank_1239_rep2
1,20230325_LFX_RH_100_RH119_P1_SPEMeOH_H11_1_Pos...,medium,NaN,rep1,medium__rep1


In [5]:
#merge the metadata with the quant table grouped by 'annotation network numbers'
dfq.set_index('annotation network number', inplace=True)
dft=dfq.transpose()
dft.reset_index(inplace=True)
dft.rename(columns={'index': 'filename'}, inplace=True)
dft=pd.merge(dft, metadata_df, how ='left', on ='filename')
dft = dft.sort_values(by=['full_name'], ascending=True)
dft.head(1)

,filename,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,85x,86x,8x,96x,97x,98x,Type,Strain,Rep,full_name
38,20230325_LFX_RH_29_RH119_P1_SPEMeOH_C03_1_Pos....,0.0,0.0,0.0,113478.55,0.0,0.0,0.0,0.0,0.0,...,0.0,60738.055,0.0,0.0,0.0,0.0,blank,1016,rep1,blank_1016_rep1


In [6]:
#erase all the blanks and QC's - Change the string as needed
def drop_samples_based_on_string(df,filename,list_of_strings_for_QC_Blank_filter,column):
    print(df.shape)
    for string in list_of_strings_for_QC_Blank_filter:
        df = df[~df[column].str.contains(string, na=False)]
        df = df.dropna(how = 'any', subset=[column])
    print(df.shape)
    return df

list_of_strings_for_QC_Blank_filter = ['NaN', 'QC'] #this is the string you used to identify the samples, NaN will erase all the samples without metadata after merging.
column_to_use_for_filtering = 'Type' #this information should be included in the metadata table
dft = drop_samples_based_on_string(dft,'dft',list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)

(101, 571)
(95, 571)


In [7]:
#add a column with the same identifier for the X replicated (it'll be used to separate the table into blanks and incubations)
dft['name']=dft['Type']+'_'+dft['Strain']

#separate dataframe into blanks and incubations and obtain the mean by replicate
## blanks
dfb= dft[dft.Type == 'blank']
dfb = dfb.groupby('name', as_index=False).mean()
dfb.sort_values(by=['name'], ascending=True)
dfb.set_index('name', inplace=True)
#save file for future plot
dfbf=dfb.copy()
dfbf.reset_index(inplace=True)
dfbf.rename(columns={'name': 'Strain'}, inplace=True)
dfbf.to_csv(final_path+experiment+'_blanks_average.csv')

#replaced '0' by low vlaue
dfb = dfb.astype(int)
dfb=dfb.mask(dfb<=0, 0.00001)

##incubations
dfi= dft[dft.Type == 'incub']
dfi = dfi.groupby('name', as_index=False).mean()
dfi.sort_values(by=['name'], ascending=True)
dfi.set_index('name', inplace=True)
dfi = dfi.astype(int)

#save file for future plot
dfif=dfi.copy()
dfif.reset_index(inplace=True)
dfif.rename(columns={'name': 'Strain'}, inplace=True)
dfif.to_csv(final_path+experiment+'_incub_average.csv')

#calculate the ratio incubtation/blanks
dfr=dfi/dfb.values
dfr.head(2)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,81x,82x,83x,84x,85x,86x,8x,96x,97x,98x
name,,,,,,,,,,,,,,,,,,,,,
incub_1016,0.0,0.0,0.0,1.788141,8.954900e+09,4.304660e+10,189.507794,252.109835,1.930246e+11,1.503110e+10,...,0.0,9.865000e+09,2.509790e+10,9.917300e+10,0.0,1.378552,0.0,0.0,3.014310e+10,1.359370e+10
incub_1227,0.0,0.0,0.0,0.286335,6.174000e+09,3.397290e+10,210.920856,345.987444,1.614646e+11,9.231100e+09,...,0.0,9.370500e+09,2.426890e+10,1.039869e+11,0.0,2.346733,0.0,0.0,3.807450e+10,1.563300e+10


In [63]:
#create the table for fingerprinting based on the ratio conditions 

# change values lower than 20 by '0'
df_plot=dfr.copy().astype(int)
df_plot =df_plot.applymap(lambda x: np.nan if x >= 20 else 0)
#df1.head(2)
df_plot.update(dfi,overwrite=False)
df_plot.reset_index(inplace=True)
df_plot.rename(columns={'name': 'Strain'}, inplace=True)
#df1.head(2)
#check dataframe shape
#df1.shape

#drop columns containing only zeros and save the final file
zero_cols = [ col for col, is_zero in ((df_plot == 0).sum() == df_plot.shape[0]).items() if is_zero ]
df_plot.drop(zero_cols, axis=1, inplace=True)

df_plot.set_index('Strain')
#df_plot.reset_index(inplace=True)
df_plot.to_csv(final_path+experiment+'_incub_df_to_plot.csv')
df_plot.head()

,Strain,0.0,1.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,7x,80x,81x,82x,83x,84x,85x,96x,97x,98x
0,incub_1016,0.0,0.0,89549.0,430466.0,9433698.0,11045184.0,1930246.0,150311.0,597020.0,...,459043.0,0.0,0.0,98650.0,250979.0,991730.0,0.0,0.0,301431.0,135937.0
1,incub_1227,0.0,0.0,61740.0,339729.0,7459427.0,9396327.0,1614646.0,92311.0,262780.0,...,67447.0,0.0,0.0,93705.0,242689.0,1039869.0,0.0,0.0,380745.0,156330.0
2,incub_1228,0.0,0.0,70953.0,220069.0,4768134.0,7142265.0,1224044.0,213523.0,1047540.0,...,0.0,0.0,0.0,361637.0,231680.0,909128.0,117244.0,0.0,304513.0,122326.0
3,incub_1230,0.0,0.0,1009036.0,238829.0,5143362.0,6505800.0,1192983.0,361753.0,307416.0,...,0.0,0.0,0.0,161492.0,0.0,2121617.0,0.0,0.0,430959.0,0.0
4,incub_1233,45626.0,0.0,915043.0,287711.0,6458836.0,7457517.0,1347877.0,373426.0,0.0,...,0.0,31778.0,41952.0,159342.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2) create the fingerprints

In [64]:
#Plot the treated data for the incubations
df_plot =pd.read_csv(final_path+experiment+'_incub_df_to_plot.csv')
df_plot.drop(['Unnamed: 0'], axis=1, inplace=True)

#create and write fig
fig = px.bar(df_plot, y="Strain", x=df_plot.columns[1:], 
                orientation="h", template="simple_white", 
                width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+'Fingerprint_incub_treated.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
fig.show()

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/plotly/express/_core.py:1222: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [78]:
#Plot the RAW data for the incubations
df_incub =pd.read_csv(final_path+experiment+'_incub_average.csv')
df_incub.drop(['Unnamed: 0'], axis=1, inplace=True)

#create and write fig
fig = px.bar(df_incub, y="Strain", x=df_incub.columns[1:], orientation="h", template="simple_white", width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+'Fingerprint_incub_raw.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
fig.show()

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/plotly/express/_core.py:1222: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [79]:
#Plot the RAW data for the Blanks
df_blank=pd.read_csv(final_path+experiment+'_blanks_average.csv')
df_blank.drop(['Unnamed: 0'], axis=1, inplace=True)

#create and write fig
fig = px.bar(df_blank, y="Strain", x=df_blank.columns[1:], orientation="h", template="simple_white", width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+'Fingerprint_blank_raw.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
fig.show()

/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/plotly/express/_core.py:1222: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

